In [2]:
from pickle import load

from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.inception_v3 import InceptionV3

from keras_preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from keras.applications.inception_v3 import preprocess_input as inception_preprocess

from keras_preprocessing.sequence import pad_sequences
from numpy import argmax

from matplotlib import image as mpimg, pyplot as plt

In [4]:
PATH_TOKENIZER = "tokenizer.pkl"
PATH_VGG16 = "bleu-0512229-vgg16.h5"
PATH_INCEPTION = "bleu-0507810-inception.h5"

TOKENIZER = load(open(PATH_TOKENIZER, 'rb'))
MAX_WORDS = 22

VGG16_EXTRACTOR = VGG16(weights="imagenet")
VGG16_EXTRACTOR = Model(inputs=VGG16_EXTRACTOR.inputs, outputs=VGG16_EXTRACTOR.layers[-2].output)
VGG16_TARGET_SIZE = 224
VGG16_MODEL = load_model(PATH_VGG16)

INCEPTION_EXTRACTOR = InceptionV3(weights="imagenet")
INCEPTION_EXTRACTOR = Model(inputs=INCEPTION_EXTRACTOR.inputs, outputs=INCEPTION_EXTRACTOR.layers[-2].output)
INCEPTION_TARGET_SIZE = 299
INCEPTION_MODEL = load_model(PATH_INCEPTION)